In [63]:
import os
import re
import itertools
from collections import Counter
import math
import pickle

from itertools import product
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV, cross_validate
from sklearn.linear_model import  LinearRegression, Ridge, Lasso, RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.inspection import permutation_importance
import joblib
import matplotlib.pyplot as plt

import xgboost as xgb
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

from CAI import CAI

In [2]:
def safe_float_from_cai_obj(obj):
    try:
        return float(obj)
    except Exception:
        pass
    if hasattr(obj, "cai"):
        return float(obj.cai)
    if hasattr(obj, "value"):
        return float(obj.value)
    raise ValueError("Unknown CAI object -> can't extract float")

def_refs = [
    "ATGGATGATGATATCGCCGCGCTCGTCGTCGACAACGGCTCCGGCATGTGCAAGGCCGGCTTCGCGGGCGACGATGCCCCCCGGGCCGTCTTCCCCTCCATCGTGGGGCGCCCCAGGCACCAGGGCGTGATGGTGGGCATGGGTCAGAAGGATTCCTATGTGGGCGACGAGGCCCAGAGCAAGAGAGGCATCCTCACCCTGAAGTACCCCATCGAGCACGGCATCGTCACCAACTGGGACGACATGGAGAAAATCTGGCACCACACCTTCTACAATGAGCTGCGTGTGGCTCCCGAGGAGCACCCCGTGCTGCTGACCGAGGCCCCCCTGAACCCCAAGGCCAACCGCGAGAAGATGACCCAGATCATGTTTGAGACCTTCAACACCCCAGCCATGTACGTTGCTATCCAGGCTGTGCTATCCCTGTACGCCTCTGGCCGTACCACTGGCATCGTGATGGACTCCGGTGACGGGGTCACCCACACTGTGCCCATCTACGAGGGGTATGCCCTCCCCCATGCCATCCTGCGTCTGGACCTGGCTGGCCGGGACCTGACTGACTACCTCATGAAGATCCTCACCGAGCGCGGCTACAGCTTCACCACCACGGCCGAGCGGGAAATCGTGCGTGACATTAAGGAGAAGCTGTGCTACGTCGCCCTGGACTTCGAGCAAGAGATGGCCACGGCTGCTTCCAGCTCCTCCCTGGAGAAGAGCTACGAGCTGCCTGACGGCCAGGTCATCACCATTGGCAATGAGCGGTTCCGCTGCCCTGAGGCACTCTTCCAGCCTTCCTTCCTGGGCATGGAGTCCTGTGGCATCCACGAAACTACCTTCAACTCCATCATGAAGTGTGACGTGGACATCCGCAAAGACCTGTACGCCAACACAGTGCTGTCTGGCGGCACCACCATGTACCCTGGCATTGCCGACAGGATGCAGAAGGAGATCACTGCCCTGGCACCCAGCACAATGAAGATCAAGATCATTGCTCCTCCTGAGCGCAAGTACTCCGTGTGGATCGGCGGCTCCATCCTGGCCTCGCTGTCCACCTTCCAGCAGATGTGGATCAGCAAGCAGGAGTATGACGAGTCCGGCCCCTCCATCGTCCACCGCAAATGCTTCTAG \n ATGGGGAAGAACAAACTCCTTCATCCAAGTCTGGTTCTTCTCCTCTTGGTCCTCCTGCCCACAGACGCCTCAGTCTCTGGAAAACCGCAGTATATGGTTCTGGTCCCCTCCCTGCTCCACACTGAGACCACTGAGAAGGGCTGTGTCCTTCTGAGCTACCTGAATGAGACAGTGACTGTAAGTGCTTCCTTGGAGTCTGTCAGGGGAAACAGGAGCCTCTTCACTGACCTGGAGGCGGAGAATGACGTACTCCACTGTGTCGCCTTCGCTGTCCCAAAGTCTTCATCCAATGAGGAGGTAATGTTCCTCACTGTCCAAGTGAAAGGACCAACCCAAGAATTTAAGAAGCGGACCACAGTGATGGTTAAGAACGAGGACAGTCTGGTCTTTGTCCAGACAGACAAATCAATCTACAAACCAGGGCAGACAGTGAAATTTCGTGTTGTCTCCATGGATGAAAACTTTCACCCCCTGAATGAGTTGATTCCACTAGTATACATTCAGGATCCCAAAGGAAATCGCATCGCACAATGGCAGAGTTTCCAGTTAGAGGGTGGCCTCAAGCAATTTTCTTTTCCCCTCTCATCAGAGCCCTTCCAGGGCTCCTACAAGGTGGTGGTACAGAAGAAATCAGGTGGAAGGACAGAGCACCCTTTCACCGTGGAGGAATTTGTTCTTCCCAAGTTTGAAGTACAAGTAACAGTGCCAAAGATAATCACCATCTTGGAAGAAGAGATGAATGTATCAGTGTGTGGCCTATACACATATGGGAAGCCTGTCCCTGGACATGTGACTGTGAGCATTTGCAGAAAGTATAGTGACGCTTCCGACTGCCACGGTGAAGATTCACAGGCTTTCTGTGAGAAATTCAGTGGACAGCTAAACAGCCATGGCTGCTTCTATCAGCAAGTAAAAACCAAGGTCTTCCAGCTGAAGAGGAAGGAGTATGAAATGAAACTTCACACTGAGGCCCAGATCCAAGAAGAAGGAACAGTGGTGGAATTGACTGGAAGGCAGTCCAGTGAAATCACAAGAACCATAACCAAACTCTCATTTGTGAAAGTGGACTCACACTTTCGACAGGGAATTCCCTTCTTTGGGCAGGTGCGCCTAGTAGATGGGAAAGGCGTCCCTATACCAAATAAAGTCATATTCATCAGAGGAAATGAAGCAAACTATTACTCCAATGCTACCACGGATGAGCATGGCCTTGTACAGTTCTCTATCAACACCACCAATGTTATGGGTACCTCTCTTACTGTTAGGGTCAATTACAAGGATCGTAGTCCCTGTTACGGCTACCAGTGGGTGTCAGAAGAACACGAAGAGGCACATCACACTGCTTATCTTGTGTTCTCCCCAAGCAAGAGCTTTGTCCACCTTGAGCCCATGTCTCATGAACTACCCTGTGGCCATACTCAGACAGTCCAGGCACATTATATTCTGAATGGAGGCACCCTGCTGGGGCTGAAGAAGCTCTCCTTCTATTATCTGATAATGGCAAAGGGAGGCATTGTCCGAACTGGGACTCATGGACTGCTTGTGAAGCAGGAAGACATGAAGGGCCATTTTTCCATCTCAATCCCTGTGAAGTCAGACATTGCTCCTGTCGCTCGGTTGCTCATCTATGCTGTTTTACCTACCGGGGACGTGATTGGGGATTCTGCAAAATATGATGTTGAAAATTGTCTGGCCAACAAGGTGGATTTGAGCTTCAGCCCATCACAAAGTCTCCCAGCCTCACACGCCCACCTGCGAGTCACAGCGGCTCCTCAGTCCGTCTGCGCCCTCCGTGCTGTGGACCAAAGCGTGCTGCTCATGAAGCCTGATGCTGAGCTCTCGGCGTCCTCGGTTTACAACCTGCTACCAGAAAAGGACCTCACTGGCTTCCCTGGGCCTTTGAATGACCAGGACAATGAAGACTGCATCAATCGTCATAATGTCTATATTAATGGAATCACATATACTCCAGTATCAAGTACAAATGAAAAGGATATGTACAGCTTCCTAGAGGACATGGGCTTAAAGGCATTCACCAACTCAAAGATTCGTAAACCCAAAATGTGTCCACAGCTTCAACAGTATGAAATGCATGGACCTGAAGGTCTACGTGTAGGTTTTTATGAGTCAGATGTAATGGGAAGAGGCCATGCACGCCTGGTGCATGTTGAAGAGCCTCACACGGAGACCGTACGAAAGTACTTCCCTGAGACATGGATCTGGGATTTGGTGGTGGTAAACTCAGCAGGTGTGGCTGAGGTAGGAGTAACAGTCCCTGACACCATCACCGAGTGGAAGGCAGGGGCCTTCTGCCTGTCTGAAGATGCTGGACTTGGTATCTCTTCCACTGCCTCTCTCCGAGCCTTCCAGCCCTTCTTTGTGGAGCTCACAATGCCTTACTCTGTGATTCGTGGAGAGGCCTTCACACTCAAGGCCACGGTCCTAAACTACCTTCCCAAATGCATCCGGGTCAGTGTGCAGCTGGAAGCCTCTCCCGCCTTCCTAGCTGTCCCAGTGGAGAAGGAACAAGCGCCTCACTGCATCTGTGCAAACGGGCGGCAAACTGTGTCCTGGGCAGTAACCCCAAAGTCATTAGGAAATGTGAATTTCACTGTGAGCGCAGAGGCACTAGAGTCTCAAGAGCTGTGTGGGACTGAGGTGCCTTCAGTTCCTGAACACGGAAGGAAAGACACAGTCATCAAGCCTCTGTTGGTTGAACCTGAAGGACTAGAGAAGGAAACAACATTCAACTCCCTACTTTGTCCATCAGGTGGTGAGGTTTCTGAAGAATTATCCCTGAAACTGCCACCAAATGTGGTAGAAGAATCTGCCCGAGCTTCTGTCTCAGTTTTGGGAGACATATTAGGCTCTGCCATGCAAAACACACAAAATCTTCTCCAGATGCCCTATGGCTGTGGAGAGCAGAATATGGTCCTCTTTGCTCCTAACATCTATGTACTGGATTATCTAAATGAAACACAGCAGCTTACTCCAGAGATCAAGTCCAAGGCCATTGGCTATCTCAACACTGGTTACCAGAGACAGTTGAACTACAAACACTATGATGGCTCCTACAGCACCTTTGGGGAGCGATATGGCAGGAACCAGGGCAACACCTGGCTCACAGCCTTTGTTCTGAAGACTTTTGCCCAAGCTCGAGCCTACATCTTCATCGATGAAGCACACATTACCCAAGCCCTCATATGGCTCTCCCAGAGGCAGAAGGACAATGGCTGTTTCAGGAGCTCTGGGTCACTGCTCAACAATGCCATAAAGGGAGGAGTAGAAGATGAAGTGACCCTCTCCGCCTATATCACCATCGCCCTTCTGGAGATTCCTCTCACAGTCACTCACCCTGTTGTCCGCAATGCCCTGTTTTGCCTGGAGTCAGCCTGGAAGACAGCACAAGAAGGGGACCATGGCAGCCATGTATATACCAAAGCACTGCTGGCCTATGCTTTTGCCCTGGCAGGTAACCAGGACAAGAGGAAGGAAGTACTCAAGTCACTTAATGAGGAAGCTGTGAAGAAAGACAACTCTGTCCATTGGGAGCGCCCTCAGAAACCCAAGGCACCAGTGGGGCATTTTTACGAACCCCAGGCTCCCTCTGCTGAGGTGGAGATGACATCCTATGTGCTCCTCGCTTATCTCACGGCCCAGCCAGCCCCAACCTCGGAGGACCTGACCTCTGCAACCAACATCGTGAAGTGGATCACGAAGCAGCAGAATGCCCAGGGCGGTTTCTCCTCCACCCAGGACACAGTGGTGGCTCTCCATGCTCTGTCCAAATATGGAGCAGCCACATTTACCAGGACTGGGAAGGCTGCACAGGTGACTATCCAGTCTTCAGGGACATTTTCCAGCAAATTCCAAGTGGACAACAACAACCGCCTGTTACTGCAGCAGGTCTCATTGCCAGAGCTGCCTGGGGAATACAGCATGAAAGTGACAGGAGAAGGATGTGTCTACCTCCAGACATCCTTGAAATACAATATTCTCCCAGAAAAGGAAGAGTTCCCCTTTGCTTTAGGAGTGCAGACTCTGCCTCAAACTTGTGATGAACCCAAAGCCCACACCAGCTTCCAAATCTCCCTAAGTGTCAGTTACACAGGGAGCCGCTCTGCCTCCAACATGGCGATCGTTGATGTGAAGATGGTCTCTGGCTTCATTCCCCTGAAGCCAACAGTGAAAATGCTTGAAAGATCTAACCATGTGAGCCGGACAGAAGTCAGCAGCAACCATGTCTTGATTTACCTTGATAAGGTGTCAAATCAGACACTGAGCTTGTTCTTCACGGTTCTGCAAGATGTCCCAGTAAGAGATCTGAAACCAGCCATAGTGAAAGTCTATGATTACTACGAGACGGATGAGTTTGCAATTGCTGAGTACAATGCTCCTTGCAGCAAAGATCTTGGAAATGCTTGA",
    "ATGGACCATTATGATTCTCAGCAAACCAACGATTACATGCAGCCAGAAGAGGACTGGGACCGGGACCTGCTCCTGGACCCGGCCTGGGAGAAGCAGCAGAGAAAGACATTCACGGCATGGTGTAACTCCCACCTCCGGAAGGCGGGGACACAGATCGAGAACATCGAAGAGGACTTCCGGGATGGCCTGAAGCTCATGCTGCTGCTGGAGGTCATCTCAGGTGAACGCTTGGCCAAGCCAGAGCGAGGCAAGATGAGAGTGCACAAGATCTCCAACGTCAACAAGGCCCTGGATTTCATAGCCAGCAAAGGCGTCAAACTGGTGTCCATCGGAGCCGAAGAAATCGTGGATGGGAATGTGAAGATGACCCTGGGCATGATCTGGACCATCATCCTGCGCTTTGCCATCCAGGACATCTCCGTGGAAGAGACTTCAGCCAAGGAAGGGCTGCTCCTGTGGTGTCAGAGAAAGACAGCCCCTTACAAAAATGTCAACATCCAGAACTTCCACATAAGCTGGAAGGATGGCCTCGGCTTCTGTGCTTTGATCCACCGACACCGGCCCGAGCTGATTGACTACGGGAAGCTGCGGAAGGATGATCCACTCACAAATCTGAATACGGCTTTTGACGTGGCAGAGAAGTACCTGGACATCCCCAAGATGCTGGATGCCGAAGACATCGTTGGAACTGCCCGACCGGATGAGAAAGCCATCATGACTTACGTGTCTAGCTTCTACCACGCCTTCTCTGGAGCCCAGAAGGCGGAGACAGCAGCCAATCGCATCTGCAAGGTGTTGGCCGTCAACCAGGAGAACGAGCAGCTTATGGAAGACTACGAGAAGCTGGCCAGTGATCTGTTGGAGTGGATCCGCCGCACAATCCCGTGGCTGGAGAACCGGGTGCCCGAGAACACCATGCATGCCATGCAACAGAAGCTGGAGGACTTCCGGGACTACCGGCGCCTGCACAAGCCGCCCAAGGTGCAGGAGAAGTGCCAGCTGGAGATCAACTTCAACACGCTGCAGACCAAGCTGCGGCTCAGCAACCGGCCTGCCTTCATGCCCTCTGAGGGCAGGATGGTCTCGGACATCAACAATGCCTGGGGCTGCCTGGAGCAGGTGGAGAAGGGCTATGAGGAGTGGTTGCTGAATGAGATCCGGAGGCTGGAGCGACTGGACCACCTGGCAGAGAAGTTCCGGCAGAAGGCCTCCATCCACGAGGCCTGGACTGACGGCAAAGAGGCCATGCTGCGACAGAAGGACTATGAGACCGCCACCCTCTCGGAGATCAAGGCCCTGCTCAAGAAGCATGAGGCCTTCGAGAGTGACCTGGCTGCCCACCAGGACCGTGTGGAGCAGATTGCCGCCATCGCACAGGAGCTCAATGAGCTGGACTATTATGACTCACCCAGTGTCAACGCCCGTTGCCAAAAGATCTGTGACCAGTGGGACAATCTGGGGGCCCTAACTCAGAAGCGAAGGGAAGCTCTGGAGCGGACCGAGAAACTGCTGGAGACCATTGACCAGCTGTACTTGGAGTATGCCAAGCGGGCTGCACCCTTCAACAACTGGATGGAGGGGGCCATGGAGGACCTGCAGGACACCTTCATTGTGCACACCATTGAGGAGATCCAGGGACTGACCACAGCCCATGAGCAGTTCAAGGCCACCCTCCCTGATGCCGACAAGGAGCGCCTGGCCATCCTGGGCATCCACAATGAGGTGTCCAAGATTGTCCAGACCTACCACGTCAATATGGCGGGCACCAACCCCTACACAACCATCACGCCTCAGGAGATCAATGGCAAATGGGACCACGTGCGGCAGCTGGTGCCTCGGAGGGACCAAGCTCTGACGGAGGAGCATGCCCGACAGCAGCACAATGAGAGGCTACGCAAGCAGTTTGGAGCCCAGGCCAATGTCATCGGGCCCTGGATCCAGACCAAGATGGAGGAGATCGGGAGGATCTCCATTGAGATGCATGGGACCCTGGAGGACCAGCTCAGCCACCTGCGGCAGTATGAGAAGAGCATCGTCAACTACAAGCCAAAGATTGATCAGCTGGAGGGCGACCACCAGCTCATCCAGGAGGCGCTCATCTTCGACAACAAGCACACCAACTACACCATGGAGCACATCCGTGTGGGCTGGGAGCAGCTGCTCACCACCATCGCCAGGACCATCAATGAGGTAGAGAACCAGATCCTGACCCGGGATGCCAAGGGCATCAGCCAGGAGCAGATGAATGAGTTCCGGGCCTCCTTCAACCACTTTGACCGGGATCACTCCGGCACACTGGGTCCCGAGGAGTTCAAAGCCTGCCTCATCAGCTTGGGTTATGATATTGGCAACGACCCCCAGGGAGAAGCAGAATTTGCCCGCATCATGAGCATTGTGGACCCCAACCGCCTGGGGGTAGTGACATTCCAGGCCTTCATTGACTTCATGTCCCGCGAGACAGCCGACACAGATACAGCAGACCAAGTCATGGCTTCCTTCAAGATCCTGGCTGGGGACAAGAACTACATTACCATGGACGAGCTGCGCCGCGAGCTGCCACCCGACCAGGCTGAGTACTGCATCGCGCGGATGGCCCCCTACACCGGCCCCGACTCCGTGCCAGGTGCTCTGGACTACATGTCCTTCTCCACGGCGCTGTACGGCGAGAGTGACCTCTAA",
    "ATGTCGCTTTCTAACAAGCTGACGCTGGACAAGCTGGACGTTAAAGGGAAGCGGGTCGTTATGAGAGTCGACTTCAATGTTCCTATGAAGAACAACCAGATAACAAACAACCAGAGGATTAAGGCTGCTGTCCCAAGCATCAAATTCTGCTTGGACAATGGAGCCAAGTCGGTAGTCCTTATGAGCCACCTAGGCCGGCCTGATGGTGTGCCCATGCCTGACAAGTACTCCTTAGAGCCAGTTGCTGTAGAACTCAAATCTCTGCTGGGCAAGGATGTTCTGTTCTTGAAGGACTGTGTAGGCCCAGAAGTGGAGAAAGCCTGTGCCAACCCAGCTGCTGGGTCTGTCATCCTGCTGGAGAACCTCCGCTTTCATGTGGAGGAAGAAGGGAAGGGAAAAGATGCTTCTGGGAACAAGGTTAAAGCCGAGCCAGCCAAAATAGAAGCTTTCCGAGCTTCACTTTCCAAGCTAGGGGATGTCTATGTCAATGATGCTTTTGGCACTGCTCACAGAGCCCACAGCTCCATGGTAGGAGTCAATCTGCCACAGAAGGCTGGTGGGTTTTTGATGAAGAAGGAGCTGAACTACTTTGCAAAGGCCTTGGAGAGCCCAGAGCGACCCTTCCTGGCCATCCTGGGCGGAGCTAAAGTTGCAGACAAGATCCAGCTCATCAATAATATGCTGGACAAAGTCAATGAGATGATTATTGGTGGTGGAATGGCTTTTACCTTCCTTAAGGTGCTCAACAACATGGAGATTGGCACTTCTCTGTTTGATGAAGAGGGAGCCAAGATTGTCAAAGACCTAATGTCCAAAGCTGAGAAGAATGGTGTGAAGATTACCTTGCCTGTTGACTTTGTCACTGCTGACAAGTTTGATGAGAATGCCAAGACTGGCCAAGCCACTGTGGCTTCTGGCATACCTGCTGGCTGGATGGGCTTGGACTGTGGTCCTGAAAGCAGCAAGAAGTATGCTGAGGCTGTCACTCGGGCTAAGCAGATTGTGTGGAATGGTCCTGTGGGGGTATTTGAATGGGAAGCTTTTGCCCGGGGAACCAAAGCTCTCATGGATGAGGTGGTGAAAGCCACTTCTAGGGGCTGCATCACCATCATAGGTGGTGGAGACACTGCCACTTGCTGTGCCAAATGGAACACGGAGGATAAAGTCAGCCATGTGAGCACTGGGGGTGGTGCCAGTTTGGAGCTCCTGGAAGGTAAAGTCCTTCCTGGGGTGGATGCTCTCAGCAATATTTAG",
    "ATGCCCAGGGAAGACAGGGCGACCTGGAAGTCCAACTACTTCCTTAAGATCATCCAACTATTGGATGATTATCCGAAATGTTTCATTGTGGGAGCAGACAATGTGGGCTCCAAGCAGATGCAGCAGATCCGCATGTCCCTTCGCGGGAAGGCTGTGGTGCTGATGGGCAAGAACACCATGATGCGCAAGGCCATCCGAGGGCACCTGGAAAACAACCCAGCTCTGGAGAAACTGCTGCCTCATATCCGGGGGAATGTGGGCTTTGTGTTCACCAAGGAGGACCTCACTGAGATCAGGGACATGTTGCTGGCCAATAAGGTGCCAGCTGCTGCCCGTGCTGGTGCCATTGCCCCATGTGAAGTCACTGTGCCAGCCCAGAACACTGGTCTCGGGCCCGAGAAGACCTCCTTTTTCCAGGCTTTAGGTATCACCACTAAAATCTCCAGGGGCACCATTGAAATCCTGAGTGATGTGCAGCTGATCAAGACTGGAGACAAAGTGGGAGCCAGCGAAGCCACGCTGCTGAACATGCTCAACATCTCCCCCTTCTCCTTTGGGCTGGTCATCCAGCAGGTGTTCGACAATGGCAGCATCTACAACCCTGAAGTGCTTGATATCACAGAGGAAACTCTGCATTCTCGCTTCCTGGAGGGTGTCCGCAATGTTGCCAGTGTCTGTCTGCAGATTGGCTACCCAACTGTTGCATCAGTACCCCATTCTATCATCAACGGGTACAAACGAGTCCTGGCCTTGTCTGTGGAGACGGATTACACCTTCCCACTTGCTGAAAAGGTCAAGGCCTTCTTGGCTGATCCATCTGCCTTTGTGGCTGCTGCCCCTGTGGCTGCTGCCACCACAGCTGCTCCTGCTGCTGCTGCAGCCCCAGCTAAGGTTGAAGCCAAGGAAGAGTCGGAGGAGTCGGACGAGGATATGGGATTTGGTCTCTTTGACTAA",
    "ATGGGGAAGGTGAAGGTCGGAGTCAACGGATTTGGTCGTATTGGGCGCCTGGTCACCAGGGCTGCTTTTAACTCTGGTAAAGTGGATATTGTTGCCATCAATGACCCCTTCATTGACCTCAACTACATGGTTTACATGTTCCAATATGATTCCACCCATGGCAAATTCCATGGCACCGTCAAGGCTGAGAACGGGAAGCTTGTCATCAATGGAAATCCCATCACCATCTTCCAGGAGCGAGATCCCTCCAAAATCAAGTGGGGCGATGCTGGCGCTGAGTACGTCGTGGAGTCCACTGGCGTCTTCACCACCATGGAGAAGGCTGGGGCTCATTTGCAGGGGGGAGCCAAAAGGGTCATCATCTCTGCCCCCTCTGCTGATGCCCCCATGTTCGTCATGGGTGTGAACCATGAGAAGTATGACAACAGCCTCAAGATCATCAGCAATGCCTCCTGCACCACCAACTGCTTAGCACCCCTGGCCAAGGTCATCCATGACAACTTTGGTATCGTGGAAGGACTCATGACCACAGTCCATGCCATCACTGCCACCCAGAAGACTGTGGATGGCCCCTCCGGGAAACTGTGGCGTGATGGCCGCGGGGCTCTCCAGAACATCATCCCTGCCTCTACTGGCGCTGCCAAGGCTGTGGGCAAGGTCATCCCTGAGCTGAACGGGAAGCTCACTGGCATGGCCTTCCGTGTCCCCACTGCCAACGTGTCAGTGGTGGACCTGACCTGCCGTCTAGAAAAACCTGCCAAATATGATGACATCAAGAAGGTGGTGAAGCAGGCGTCGGAGGGCCCCCTCAAGGGCATCCTGGGCTACACTGAGCACCAGGTGGTCTCCTCTGACTTCAACAGCGACACCCACTCCTCCACCTTTGACGCTGGGGCTGGCATTGCCCTCAACGACCACTTTGTCAAGCTCATTTCCTGGTATGACAACGAATTTGGCTACAGCAACAGGGTGGTGGACCTCATGGCCCACATGGCCTCCAAGGAGTAA",
    "ATGGTTTACATGTTCCAATATGATTCCACCCATGGCAAATTCCATGGCACCGTCAAGGCTGAGAACGGGAAGCTTGTCATCAATGGAAATCCCATCACCATCTTCCAGGAGCGAGATCCCTCCAAAATCAAGTGGGGCGATGCTGGCGCTGAGTACGTCGTGGAGTCCACTGGCGTCTTCACCACCATGGAGAAGGCTGGGGCTCATTTGCAGGGGGGAGCCAAAAGGGTCATCATCTCTGCCCCCTCTGCTGATGCCCCCATGTTCGTCATGGGTGTGAACCATGAGAAGTATGACAACAGCCTCAAGATCATCAGCAATGCCTCCTGCACCACCAACTGCTTAGCACCCCTGGCCAAGGTCATCCATGACAACTTTGGTATCGTGGAAGGACTCATGACCACAGTCCATGCCATCACTGCCACCCAGAAGACTGTGGATGGCCCCTCCGGGAAACTGTGGCGTGATGGCCGCGGGGCTCTCCAGAACATCATCCCTGCCTCTACTGGCGCTGCCAAGGCTGTGGGCAAGGTCATCCCTGAGCTGAACGGGAAGCTCACTGGCATGGCCTTCCGTGTCCCCACTGCCAACGTGTCAGTGGTGGACCTGACCTGCCGTCTAGAAAAACCTGCCAAATATGATGACATCAAGAAGGTGGTGAAGCAGGCGTCGGAGGGCCCCCTCAAGGGCATCCTGGGCTACACTGAGCACCAGGTGGTCTCCTCTGACTTCAACAGCGACACCCACTCCTCCACCTTTGACGCTGGGGCTGGCATTGCCCTCAACGACCACTTTGTCAAGCTCATTTCCTGGTATGACAACGAATTTGGCTACAGCAACAGGGTGGTGGACCTCATGGCCCACATGGCCTCCAAGGAGTAA",
    "ATGGTGCATCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAACTTCAGGCTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAA",
    "ATGGTGCTGTCTCCTGCCGACAAGACCAACGTCAAGGCCGCCTGGGGTAAGGTCGGCGCGCACGCTGGCGAGTATGGTGCGGAGGCCCTGGAGAGGATGTTCCTGTCCTTCCCCACCACCAAGACCTACTTCCCGCACTTCGACCTGAGCCACGGCTCTGCCCAGGTTAAGGGCCACGGCAAGAAGGTGGCCGACGCGCTGACCAACGCCGTGGCGCACGTGGACGACATGCCCAACGCGCTGTCCGCCCTGAGCGACCTGCACGCGCACAAGCTTCGGGTGGACCCGGTCAACTTCAAGCTCCTAAGCCACTGCCTGCTGGTGACCCTGGCCGCCCACCTCCCCGCCGAGTTCACCCCTGCGGTGCACGCCTCCCTGGACAAGTTCCTGGCTTCTGTGAGCACCGTGCTGACCTCCAAATACCGTTAA",
    "ATGGTGCATCTGACTCCTGAGGAGAAGACTGCTGTCAATGCCCTGTGGGGCAAAGTGAACGTGGATGCAGTTGGTGGTGAGGCCCTGGGCAGATTACTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCTCTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAGGTGCTAGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACTTTTTCTCAGCTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAACTTCAGGCTCTTGGGCAATGTGCTGGTGTGTGTGCTGGCCCGCAACTTTGGCAAGGAATTCACCCCACAAATGCAGGCTGCCTATCAGAAGGTGGTGGCTGGTGTGGCTAATGCCCTGGCTCACAAGTACCATTGA",
]


In [3]:
class CAICalculator:
    def __init__(self, target_rna: str):
        self.target_rna = target_rna
        self.target_dna = target_rna.replace("U", "T")
        self.reference_seqs = def_refs

    def _load_reference_sequences_from_file(self, filepath: str) -> list[str]:
        with open(filepath, "r") as f:
            return [line.strip() for line in f if line.strip()]

    def compute_cai(self) -> float:
        cai_obj = CAI(self.target_dna, reference=self.reference_seqs)
        return safe_float_from_cai_obj(cai_obj)

In [4]:
NUCLEOTIDES = ["A", "C", "G", "T"]
CODONS = [''.join(c) for c in itertools.product(NUCLEOTIDES, repeat=3)]

In [ ]:
def clean_dna(seq: str) -> str:
    seq = seq.upper().replace(" ", "").replace("\n", "")
    seq = seq.replace("U", "T")
    seq = re.sub(r'[^ACGT]', '', seq)
    return seq

def codon_list_from_dna(seq: str):
    seq = clean_dna(seq)
    L = (len(seq) // 3) * 3
    seq = seq[:L]
    return [seq[i:i+3] for i in range(0, len(seq), 3)]


# A vector with frequency counts for each codon (triplet of nucleotides, e.g., ATG, GGC, etc.).
def codon_freqs_vector(seq: str):
    codons = codon_list_from_dna(seq)
    c = Counter(codons)
    total = max(1, len(codons))
    return np.array([c.get(codon, 0) / total for codon in CODONS], dtype=float)

# Compute :
# gc: GC content (% of G and C nucleotides in the DNA).
# gc3: GC content specifically at the third codon position.
# nt_length: Length of the nucleotide sequence.

def gc_content(seq: str):
    seq = clean_dna(seq)
    if len(seq) == 0: return 0.0
    return (seq.count("G") + seq.count("C")) / len(seq)

def gc3_content(seq: str):
    codons = codon_list_from_dna(seq)
    if len(codons) == 0:
        return 0.0
    thirds = [codon[2] for codon in codons]
    return (thirds.count("G") + thirds.count("C")) / len(thirds)

def nt_length(seq: str):
    return len(clean_dna(seq))

def aa_length_from_protein(protein_seq: str):
    if not isinstance(protein_seq, str):
        return 0
    p = protein_seq.replace(" ", "").replace("\n", "")
    p = p.replace("*", "")
    return len(p)

# Frequency of each amino acid (A, C, D, …, Y).
def aa_composition_vector(protein_seq: str):
    aa_alphabet = list("ACDEFGHIKLMNPQRSTVWY")
    p = (protein_seq or "").upper().replace(" ", "").replace("*", "")
    total = max(1, len(p))
    c = Counter(p)
    return np.array([c.get(aa, 0) / total for aa in aa_alphabet], dtype=float)

Compute a set of sequence-based features / Sequences are now encoded into feature vectors.

CAI (Codon Adaptation Index) is a number that depends on codon usage. To predict CAI, we don’t need the literal DNA string, we need codon frequencies and related features. The features this function builds (gc, gc3, codon frequencies, AA composition, etc.) are exactly the type of inputs we need for regression.

In [19]:
def build_feature_dataframe(df: pd.DataFrame, coding_col="coding_sequence", protein_col="protein_sequence"):

    feats = []
    for _, row in df.iterrows():
        coding = row.get(coding_col, "")
        coding_clean = clean_dna(coding)
        codon_vec = codon_freqs_vector(coding_clean)
        gc = gc_content(coding_clean)
        gc3 = gc3_content(coding_clean)
        ntlen = nt_length(coding_clean)
        aal = aa_length_from_protein(row.get(protein_col, ""))

        aa_comp = aa_composition_vector(row.get(protein_col, "")) if protein_col in df.columns else np.zeros(20)
        feat = {
            "gc": gc,
            "gc3": gc3,
            "nt_length": ntlen,
            "aa_length": aal,
        }
        codon_dict = {f"codon_{codon}": val for codon, val in zip(CODONS, codon_vec)}
        aa_dict = {f"aa_{aa}": val for aa, val in zip(list("ACDEFGHIKLMNPQRSTVWY"), aa_comp)}
        feat.update(codon_dict)
        feat.update(aa_dict)
        feats.append(feat)

        feat["coding_sequence"] = coding_clean
        feat["protein_sequence"] = row.get(protein_col, "")

    feat_df = pd.DataFrame(feats)
    return feat_df

In [20]:
def compute_cai_for_df(df: pd.DataFrame, coding_col="coding_sequence"):
    cai_vals = []
    for _, row in df.iterrows():
        coding = clean_dna(row.get(coding_col, ""))
        target_rna = coding.replace("T", "U")
        calc = CAICalculator(target_rna=target_rna)
        try:
            cai_val = calc.compute_cai()
        except Exception as e:
            print(f"CAI computation failed for row {_}: {e}")
            cai_val = np.nan
        cai_vals.append(cai_val)
    return np.array(cai_vals, dtype=float)

In [21]:
DATA_PATH = "Protein_mRNA_sequence_dataset.csv"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Place your dataset named '{DATA_PATH}' in the current folder or change DATA_PATH")


print("Loading dataset...")
df = pd.read_csv(DATA_PATH)
print(f"Loaded {len(df)} rows. Columns: {list(df.columns)}")

Loading dataset...
Loaded 3804 rows. Columns: ['protein_sequence', 'coding_sequence', 'info']


In [22]:
df

,protein_sequence,coding_sequence,info
0,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...,ATG TGC TCT CTG GGG TTG TTC CCT CCT CCA CCG CC...,NM_015665 Homo sapiens aladin WD repeat nucleo...
1,MAAGVPCALVTSCSSVFSGDQLVQHILGTEDLIVEVTSNDAVRFYP...,ATG GCT GCT GGC GTA CCC TGT GCG TTA GTC ACC AG...,NM_024666 Homo sapiens alpha and gamma adaptin...
2,MESESESGAAADTPPLETLSFHGDEEIIEVVELDPGPPDPDDLAQE...,ATG GAG TCC GAA TCG GAA AGC GGG GCT GCT GCT GA...,NM_001087 Homo sapiens angio associated migrat...
3,MAAVQMDPELAKRLFFEGATVVILNMPKGTEFGIDYNSWEVGPKFR...,ATG GCT GCC GTG CAG ATG GAT CCT GAG CTA GCC AA...,NM_015511 Homo sapiens AAR2 splicing factor (A...
4,MDSTLTASEIRQRFIDFFKRNEHTYVHSSATIPLDDPTLLFANAGM...,ATG GAC TCT ACT CTA ACA GCA AGT GAA ATC CGG CA...,NM_001605 Homo sapiens alanyl-tRNA synthetase ...
...,...,...,...
3799,MELMFAEWEDGERFSFEDSDRFEEDSLCSFISEAESLCQNWRGWRK...,ATG GAG CTG ATG TTT GCA GAG TGG GAG GAC GGA GA...,NM_015037 Homo sapiens zinc finger SWIM-type c...
3800,MASFVTEVLAHSGRLEKEDLGTRISRLTRRVEEIKGEVCNMISKKY...,ATG GCC TCG TTC GTG ACA GAA GTT TTG GCA CAC TC...,NM_004724 Homo sapiens zw10 kinetochore protei...
3801,MEIPKLLPARGTLQGGGGGGIPAGGGRVHRGPDSPAGQVPTRRLLL...,ATG GAA ATC CCG AAG CTG CTC CCG GCT CGC GGG AC...,NM_007156 Homo sapiens zinc finger X-linked du...
3802,MEIPKLLPARGTLQGGGGGGIPAGGGRVHRGPDSPAGQVPTRRLLL...,ATG GAA ATC CCG AAG CTG CTC CCG GCT CGC GGG AC...,NM_007157 Homo sapiens zinc finger X-linked du...


In [23]:
if "CAI" not in df.columns:
        print("No CAI column found - attempting to compute CAI (requires CAI package). This may take a while...")
        cai_array = compute_cai_for_df(df, coding_col="coding_sequence")
        df["CAI"] = cai_array

No CAI column found - attempting to compute CAI (requires CAI package). This may take a while...


In [26]:
initial_len = len(df)
df = df.dropna(subset=["coding_sequence"]) # ensure coding present
print(f"Dropped {initial_len - len(df)} rows without coding_sequence. Remaining: {len(df)}")

Dropped 0 rows without coding_sequence. Remaining: 3804


In [32]:
X_df = build_feature_dataframe(df, coding_col="coding_sequence", protein_col="protein_sequence")
X_df.to_csv("features_debug.csv", index=False)

X = X_df.drop(columns=["coding_sequence", "protein_sequence"])
y = df["CAI"].values

In [33]:
X_df

,gc,gc3,nt_length,aa_length,codon_AAA,codon_AAC,codon_AAG,codon_AAT,codon_ACA,codon_ACC,...,aa_P,aa_Q,aa_R,aa_S,aa_T,aa_V,aa_W,aa_Y,coding_sequence,protein_sequence
0,0.574040,0.588665,1641,546,0.007313,0.012797,0.018282,0.012797,0.018282,0.021938,...,0.097070,0.031136,0.054945,0.097070,0.062271,0.069597,0.038462,0.009158,ATGTGCTCTCTGGGGTTGTTCCCTCCTCCACCGCCTCGGGGTCAAG...,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...
1,0.452532,0.401899,948,315,0.028481,0.018987,0.018987,0.025316,0.018987,0.009494,...,0.044444,0.038095,0.028571,0.092063,0.047619,0.088889,0.019048,0.012698,ATGGCTGCTGGCGTACCCTGTGCGTTAGTCACCAGCTGCTCCTCCG...,MAAGVPCALVTSCSSVFSGDQLVQHILGTEDLIVEVTSNDAVRFYP...
2,0.585441,0.664368,1305,434,0.018391,0.011494,0.016092,0.004598,0.004598,0.039080,...,0.048387,0.034562,0.029954,0.071429,0.071429,0.092166,0.025346,0.011521,ATGGAGTCCGAATCGGAAAGCGGGGCTGCTGCTGACACCCCCCCAC...,MESESESGAAADTPPLETLSFHGDEEIIEVVELDPGPPDPDDLAQE...
3,0.567965,0.737662,1155,384,0.005195,0.020779,0.051948,0.010390,0.012987,0.023377,...,0.065104,0.041667,0.044271,0.062500,0.046875,0.065104,0.013021,0.023438,ATGGCTGCCGTGCAGATGGATCCTGAGCTAGCCAAGCGCCTCTTCT...,MAAVQMDPELAKRLFFEGATVVILNMPKGTEFGIDYNSWEVGPKFR...
4,0.532164,0.614035,2907,968,0.022704,0.019608,0.045408,0.015480,0.013416,0.013416,...,0.036157,0.043388,0.050620,0.050620,0.053719,0.070248,0.008264,0.026860,ATGGACTCTACTCTAACAGCAAGTGAAATCCGGCAGCGATTTATAG...,MDSTLTASEIRQRFIDFFKRNEHTYVHSSATIPLDDPTLLFANAGM...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,0.600832,0.638090,5529,1842,0.006511,0.015193,0.029300,0.007596,0.020619,0.013022,...,0.090119,0.038002,0.057003,0.100977,0.051574,0.054289,0.014658,0.022258,ATGGAGCTGATGTTTGCAGAGTGGGAGGACGGAGAGCGCTTCTCAT...,MELMFAEWEDGERFSFEDSDRFEEDSLCSFISEAESLCQNWRGWRK...
3800,0.425641,0.442308,2340,779,0.041026,0.017949,0.033333,0.020513,0.020513,0.011538,...,0.039795,0.052632,0.042362,0.082157,0.055199,0.059050,0.007702,0.026958,ATGGCCTCGTTCGTGACAGAAGTTTTGGCACACTCCGGGAGGCTGG...,MASFVTEVLAHSGRLEKEDLGTRISRLTRRVEEIKGEVCNMISKKY...
3801,0.602917,0.681250,2400,799,0.020000,0.015000,0.026250,0.012500,0.015000,0.020000,...,0.087610,0.046308,0.046308,0.098874,0.058824,0.043805,0.006258,0.007509,ATGGAAATCCCGAAGCTGCTCCCGGCTCGCGGGACACTACAGGGCG...,MEIPKLLPARGTLQGGGGGGIPAGGGRVHRGPDSPAGQVPTRRLLL...
3802,0.604478,0.675373,2412,803,0.021144,0.013682,0.023632,0.012438,0.017413,0.019900,...,0.090909,0.043587,0.043587,0.095890,0.062267,0.044832,0.006227,0.007472,ATGGAAATCCCGAAGCTGCTCCCGGCTCGCGGGACACTACAGGGCG...,MEIPKLLPARGTLQGGGGGGIPAGGGRVHRGPDSPAGQVPTRRLLL...


In [ ]:
X_df["codon_length"].hist(bins=50)
plt.xlabel("Codon length")
plt.ylabel("Count")
plt.title("Distribution of sequence lengths")
plt.show()

In [36]:
mask = ~np.isnan(y)
X = X[mask]
y = y[mask]
print(f"After removing NaN targets: X.shape={X.shape}, y.shape={y.shape}")

After removing NaN targets: X.shape=(3804, 88), y.shape=(3804,)


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name="Model"):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"\n=== {model_name} ===")
    print(f"MSE: {rmse:.6f}")
    print(f"MAE:  {mae:.6f}")

    print(f"R²:   {r2:.4f}")
    return model, (rmse, mae, r2)

## Regression models 

### Baseline

In [50]:
lin_reg = LinearRegression()
evaluate_model(lin_reg, X_train, y_train, X_test, y_test, "Linear Regression")


=== Linear Regression ===
MSE: 0.000030
MAE:  0.004137
R²:   0.9958


(LinearRegression(),
 (3.0337211063672234e-05, 0.004136795601920542, 0.9957544691637646))

In [51]:
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=0.001)

evaluate_model(ridge, X_train, y_train, X_test, y_test, "Ridge Regression")
evaluate_model(lasso, X_train, y_train, X_test, y_test, "Lasso Regression")


=== Ridge Regression ===
MSE: 0.000461
MAE:  0.016257
R²:   0.9354

=== Lasso Regression ===
MSE: 0.001102
MAE:  0.025923
R²:   0.8458


(Lasso(alpha=0.001),
 (0.0011017879742277823, 0.025923409543438875, 0.8458106511584162))

### Random Forest

In [52]:
rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
evaluate_model(rf, X_train, y_train, X_test, y_test, "Random Forest")


=== Random Forest ===
MSE: 0.000415
MAE:  0.015312
R²:   0.9419


(RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42),
 (0.00041541373965162673, 0.015311526723626626, 0.9418650634105674))

### XGboost

In [53]:
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
evaluate_model(xgb_model, X_train, y_train, X_test, y_test, "XGBoost")


=== XGBoost ===
MSE: 0.000201
MAE:  0.010199
R²:   0.9719


(XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, ...),
 (0.00020060937427162653, 0.010199413688594824, 0.9719257883422271))

In [56]:
# Preprocess features for NN

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

train_dataset = TensorDataset(
    torch.tensor(X_train_scaled, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)
test_dataset = TensorDataset(
    torch.tensor(X_test_scaled, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.float32)
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [57]:
class MLPRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Linear(hidden_dim//2, 1)
        )
    def forward(self, x):
        return self.net(x)

In [59]:
mlp = MLPRegressor(input_dim=X_train.shape[1])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mlp.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-3)

In [60]:
for epoch in range(100):
    mlp.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device).unsqueeze(1)
        optimizer.zero_grad()
        loss = criterion(mlp(xb), yb)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}")

Epoch 5, Train Loss: 0.0030
Epoch 10, Train Loss: 0.0015
Epoch 15, Train Loss: 0.0001
Epoch 20, Train Loss: 0.0001
Epoch 25, Train Loss: 0.0000
Epoch 30, Train Loss: 0.0001
Epoch 35, Train Loss: 0.0004
Epoch 40, Train Loss: 0.0001
Epoch 45, Train Loss: 0.0001
Epoch 50, Train Loss: 0.0001
Epoch 55, Train Loss: 0.0001
Epoch 60, Train Loss: 0.0002
Epoch 65, Train Loss: 0.0000
Epoch 70, Train Loss: 0.0001
Epoch 75, Train Loss: 0.0002
Epoch 80, Train Loss: 0.0000
Epoch 85, Train Loss: 0.0000
Epoch 90, Train Loss: 0.0001
Epoch 95, Train Loss: 0.0000
Epoch 100, Train Loss: 0.0001


In [61]:
mlp.eval()
with torch.no_grad():
    preds = mlp(torch.tensor(X_test_scaled, dtype=torch.float32).to(device)).cpu().numpy().flatten()

mse = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print("\n=== MLP ===")
print(f"MSE: {mse:.6f}, MAE: {mae:.6f}, R²: {r2:.4f}")


=== MLP ===
MSE: 0.000205, MAE: 0.011078, R²: 0.9713


### Transformer 

In [64]:
codon2idx = {c: i for i, c in enumerate(CODONS)}

def tokenize_coding(seq, max_len=300):
    """Convert DNA sequence into codon tokens (padded to max_len codons)."""
    seq = seq.upper().replace("U", "T")
    codons = [seq[i:i+3] for i in range(0, len(seq), 3)]
    token_ids = [codon2idx.get(c, 0) for c in codons]  # unknown codons -> 0
    if len(token_ids) < max_len:
        token_ids += [0] * (max_len - len(token_ids))  # pad
    else:
        token_ids = token_ids[:max_len]
    return token_ids

In [68]:
max_nt_len = X_df["nt_length"].max()
max_nt_len

13125

In [69]:
MAX_LEN = 13125
X_tokens = np.array([tokenize_coding(seq, MAX_LEN) for seq in df["coding_sequence"]])
y = df["CAI"].values

X_train_tok, X_test_tok, y_train_tok, y_test_tok = train_test_split(
    X_tokens, y, test_size=0.2, random_state=42
)

train_dataset = TensorDataset(
    torch.tensor(X_train_tok, dtype=torch.long),
    torch.tensor(y_train_tok, dtype=torch.float32)
)
test_dataset = TensorDataset(
    torch.tensor(X_test_tok, dtype=torch.long),
    torch.tensor(y_test_tok, dtype=torch.float32)
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class TransformerRegressor(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=128, dropout=0.1)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.permute(1,0,2)
        out = self.transformer(emb)
        out = out.mean(0)
        return self.fc(out)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerRegressor(vocab_size=len(CODONS), d_model=64, nhead=4, num_layers=2).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


for epoch in range(100):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device).unsqueeze(1)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}")

/home/ubuntu/ENTER/envs/mRNA_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Train Loss: 0.0011
Epoch 2, Train Loss: 0.0013
Epoch 3, Train Loss: 0.0016
Epoch 4, Train Loss: 0.0003
Epoch 5, Train Loss: 0.0005
Epoch 6, Train Loss: 0.0019
Epoch 7, Train Loss: 0.0001
Epoch 8, Train Loss: 0.0006
Epoch 9, Train Loss: 0.0023
Epoch 10, Train Loss: 0.0008
Epoch 11, Train Loss: 0.0013
Epoch 12, Train Loss: 0.0007
Epoch 13, Train Loss: 0.0003
Epoch 14, Train Loss: 0.0018
Epoch 15, Train Loss: 0.0002
Epoch 16, Train Loss: 0.0006
Epoch 17, Train Loss: 0.0022
Epoch 18, Train Loss: 0.0006
Epoch 19, Train Loss: 0.0012
Epoch 20, Train Loss: 0.0003
Epoch 21, Train Loss: 0.0004
Epoch 22, Train Loss: 0.0017
Epoch 23, Train Loss: 0.0002
Epoch 24, Train Loss: 0.0006
Epoch 25, Train Loss: 0.0010
Epoch 26, Train Loss: 0.0010


In [ ]:
# Evaluation
model.eval()
all_preds = []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy().flatten()
        all_preds.extend(preds)

mse = (mean_squared_error(y_test_tok, all_preds))
mae = mean_absolute_error(y_test_tok, all_preds)
r2 = r2_score(y_test_tok, all_preds)

print("\n=== Transformer ===")
print(f"RMSE: {mse:.6f}, MAE: {mae:.6f}, R²: {r2:.4f}")
